# Audiobooks business case

## Preprocess the data. Balance the dataset. Create 3 datasets: training, validation, and test. Save the newly created sets in a tensor friendly format (e.g. *.npz)

### Extract the data from the csv

In [9]:
import numpy as np
from sklearn import preprocessing

raw_csv_data = np.loadtxt(r'E:\Download\Audiobooks_data.csv', delimiter = ',')

unscaled_inputs_all = raw_csv_data[:,1:-1]
targets_all = raw_csv_data[:,-1]

[[9.9400e+02 1.6200e+03 1.6200e+03 1.9730e+01 1.9730e+01 1.0000e+00
  1.0000e+01 9.9000e-01 1.6038e+03 5.0000e+00 9.2000e+01 0.0000e+00]
 [1.1430e+03 2.1600e+03 2.1600e+03 5.3300e+00 5.3300e+00 0.0000e+00
  8.9100e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00 0.0000e+00]
 [2.0590e+03 2.1600e+03 2.1600e+03 5.3300e+00 5.3300e+00 0.0000e+00
  8.9100e+00 0.0000e+00 0.0000e+00 0.0000e+00 3.8800e+02 0.0000e+00]
 [2.8820e+03 1.6200e+03 1.6200e+03 5.9600e+00 5.9600e+00 0.0000e+00
  8.9100e+00 4.2000e-01 6.8040e+02 1.0000e+00 1.2900e+02 0.0000e+00]
 [3.3420e+03 2.1600e+03 2.1600e+03 5.3300e+00 5.3300e+00 0.0000e+00
  8.9100e+00 2.2000e-01 4.7520e+02 0.0000e+00 3.6100e+02 0.0000e+00]]


### Balance the dataset

In [5]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] ==0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis = 0)
targets_equal_priors = np.delete (targets_all, indices_to_remove, axis=0)

### Standardize the inputs

In [7]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Shuffle the data

In [11]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split the dataset into train, validation, and test

In [28]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.85*samples_count)
validation_samples_count = int(0.08*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1891.0 3802 0.49736980536559705
182.0 357 0.5098039215686274
164.0 315 0.5206349206349207


### Save the three datasets in *.npz

In [30]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)